# Convolutionnal model

In [0]:
import numpy as np
from random import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DATA_FILE_DIR = '/content/drive/My Drive/faces_validated.npy'
IMG_SIZE = 256

test_ratio = 0.2
validation_ratio = 0.5

We load the dataset, process the data and create our training and test sets.

In [9]:
dataset = np.load(DATA_FILE_DIR, allow_pickle = True)
print('The dataset has been loaded properly.')

OSError: ignored

In [0]:
#reshuffle the arrays
X, Y = dataset[:,0], dataset[:,1]
p = np.random.permutation(len(X))
X, Y = X[p], Y[p]

X = np.stack(X, axis=0)
nb_img = X.shape[0]
X = X.reshape((nb_img, IMG_SIZE, IMG_SIZE, 3))

Y = (Y - min(Y)) / (max(Y) - min(Y))



In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_ratio)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=validation_ratio)

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras import applications


model = applications.VGG19(weights ='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in model.layers[:20]:
    layer.trainable = False
    
x = model.output
x = Flatten()(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(1, activation='softmax')(x)

# creating the final model 
model_final = Model(inputs=model.input, outputs=prediction)

model_final.summary()

# compile the model 
model_final.compile(loss='mean_squared_error', 
                    optimizer=optimizers.SGD(lr=0.0001, momentum=0.9))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
model_final.fit(X_train, Y_train, batch_size=32, epochs=5)

Epoch 1/5
2336/6657 [=========>....................] - ETA: 1:30:11 - loss: 0.5255